In [1]:
from sklift.models import SoloModel, ClassTransformation

In [2]:
from sklift.datasets import fetch_lenta

/opt/anaconda3/envs/regularized-t-learner/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = fetch_lenta()

Lenta dataset: 100%|████████████████████████████████████████████████████| 145M/145M [01:46<00:00, 1.36MiB/s]


In [4]:
print(f"Dataset type: {type(dataset)}\n")
print(f"Dataset features shape: {dataset.data.shape}")
print(f"Dataset target shape: {dataset.target.shape}")
print(f"Dataset treatment shape: {dataset.treatment.shape}")

Dataset type: <class 'sklearn.utils._bunch.Bunch'>

Dataset features shape: (687029, 193)
Dataset target shape: (687029,)
Dataset treatment shape: (687029,)


In [5]:
dataset.data.head().append(dataset.data.tail())


/var/folders/f0/5fc11nbd3z32dmldqkrccl040000gn/T/ipykernel_56283/2476854981.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset.data.head().append(dataset.data.tail())


,age,cheque_count_12m_g20,cheque_count_12m_g21,cheque_count_12m_g25,cheque_count_12m_g32,cheque_count_12m_g33,cheque_count_12m_g38,cheque_count_12m_g39,cheque_count_12m_g41,cheque_count_12m_g42,...,sale_sum_6m_g24,sale_sum_6m_g25,sale_sum_6m_g26,sale_sum_6m_g32,sale_sum_6m_g33,sale_sum_6m_g44,sale_sum_6m_g54,stdev_days_between_visits_15d,stdev_discount_depth_15d,stdev_discount_depth_1m
0,47.0,3.0,22.0,19.0,3.0,28.0,8.0,7.0,6.0,1.0,...,3141.25,356.67,237.25,283.84,3648.23,1195.37,535.42,1.7078,0.2798,0.3008
1,57.0,1.0,0.0,2.0,1.0,1.0,1.0,0.0,1.0,0.0,...,113.39,62.69,58.71,87.01,179.83,0.00,122.98,0.0000,0.0000,0.0000
2,38.0,7.0,0.0,15.0,4.0,9.0,5.0,9.0,14.0,7.0,...,1239.19,533.46,83.37,593.13,1217.43,1336.83,3709.82,0.0000,NaN,0.0803
3,65.0,6.0,3.0,25.0,2.0,10.0,14.0,11.0,8.0,1.0,...,139.68,1849.91,360.40,175.73,496.73,172.58,1246.21,0.0000,0.0000,0.0000
4,61.0,0.0,1.0,2.0,0.0,2.0,1.0,0.0,3.0,2.0,...,226.98,168.05,461.37,0.00,237.93,225.51,995.27,1.4142,0.3495,0.3495
687024,35.0,0.0,0.0,4.0,0.0,2.0,0.0,1.0,0.0,3.0,...,550.09,669.33,111.87,0.00,330.96,1173.84,119.99,2.6458,0.3646,0.3282
687025,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,28.01,0.0000,0.0000,0.0000
687026,36.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.00,0.00,0.00,0.00,0.00,449.01,0.00,0.0000,NaN,NaN
687027,37.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.00,46.72,0.00,0.00,0.00,0.00,0.00,0.0000,NaN,NaN
687028,40.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,2.0,2.0,...,290.01,0.00,0.00,0.00,228.47,752.32,596.86,0.0000,0.0000,0.0000


In [6]:
dataset.treatment.head()

0       test
1       test
2    control
3       test
4       test
Name: group, dtype: object

In [9]:
dataset.target.head()

0    0
1    0
2    0
3    0
4    0
Name: response_att, dtype: int64

In [10]:
dataset.treatment.shape

(687029,)

In [11]:
dataset.target.shape

(687029,)